In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Load NLP modules
import re
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

In [2]:
df_business = pd.read_csv('../../data/dataset/csv/business.csv', index_col = 'business_id')
#df_checkin = pd.read_csv('../../data/dataset/csv/checkin.csv', index_col = 'business_id')
#df_photos = pd.read_csv('../../data/dataset/csv/photos.csv', index_col = 'photo_id')
df_review = pd.read_csv('../../data/dataset/csv/review.csv', index_col = 'review_id')
#df_tip = pd.read_csv('../../data/dataset/csv/tip.csv') # index not set for the df_tip
df_user = pd.read_csv('../../data/dataset/csv/user.csv', index_col = 'user_id')

In [5]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

class PreProcessing:
    pd.options.mode.chained_assignment = None  # default='warn'
    # The constructor takes a pandas dataframe as input and save it to self.df
    def __init__(self, df_r, df_b, df_u):
        self.df_r = df_r
        self.df_b = df_b
        self.df_u = df_u
        self.token_pattern = "[a-zA-Z0-9'`]+"


    def TextPreProcessing(line,
                      token_pattern=self.token_pattern):
        ## tokenize
        tokenizer = RegexpTokenizer(token_pattern)
        tokens = tokenizer.tokenize( str(line).lower() )
        return tokens

    def Myngrams(text, nfold):
        s = []
        for ngram in ngrams(text, nfold):
            s.append(' '.join(str(i) for i in ngram))

        #s = list(set(s)) # unique string in the list
        return s

    def TryDivide(x, y, val=0.0):
        """ 
        Try to divide two numbers
        """
        if y != 0.0:
            val = float(x) / y
        return val

    def GetPositionList(tgt, obs):
        """
        Get the list of positions of obs in target
        """
        pos_of_obs_in_tgt = [0]
        if len(obs) != 0:
            pos_of_obs_in_tgt = [j for j,w in enumerate(obs, start=1) if w in tgt]
            if len(pos_of_obs_in_tgt) == 0:
                pos_of_obs_in_tgt = [0]
        #print(pos_of_obs_in_tgt)
        return pos_of_obs_in_tgt

    def DumpTextBasicNgram(df):
        ## 1-gram
        print("Generate 1-gram...")
        df["text_1gram"] = list(df.apply(lambda x: Myngrams(TextPreProcessing(x["text"]), 1), axis=1))
        ## 2-gram
        print("Generate 2-gram...")
        df["text_2gram"] = list(df.apply(lambda x: Myngrams(TextPreProcessing(x["text"]), 2), axis=1))
        ## 3-gram
        print("Generate 3-gram...")
        df["text_3gram"] = list(df.apply(lambda x: Myngrams(TextPreProcessing(x["text"]), 3), axis=1))
        return ;
        
    def GenTrainDataset(self):
        #merge review and bussiness, based on review
        sub_df_b = self.df_b[['state', 'stars', 'review_count']]
        sub_df_b.rename(index=str, columns={"stars": "bstar_ave", "review_count": "breview_count"}, inplace=True)
        df_train_r_comb_b = pd.merge(self.df_r, sub_df_b, how='outer', left_on='business_id' , right_index=True)
    
        #merge review and user, based on review
        sub_df_u = self.df_u[['fans', 'review_count', 'friends']]
        sub_df_u.rename(index=str, columns={"review_count": "ureview_count"}, inplace=True)
        sub_df_u['nfriends'] = list(sub_df_u.apply(lambda x: len(x['friends']), axis=1))
        sub_df_u.drop(['friends'], axis=1, inplace=True)
        df_train_r_comb_b_u = pd.merge(df_train_r_comb_b, sub_df_u, how='outer', left_on='user_id', right_index=True)

        #build nlp features...
        
        return df_train_r_comb_b_u

preprocessing = PreProcessing(df_review, df_business, df_user)
df_train = preprocessing.GenTrainDataset()
print ("done with train set preprocessing!")
df_train.head()

done with trainning set preprocessing!


,useful,date,text,stars,business_id,user_id,cool,funny,state,bstar_ave,breview_count,fans,ureview_count,nfriends
review_id,,,,,,,,,,,,,,
v0i_UHJMo_hPBq9bxWvW4w,0,2016-05-28,"Love the staff, love the meat, love the place....",5,0W4lkclzZThpx3V65bVgig,bv2nCi5Qv5vroFiqKGopiw,0,0,QC,4.0,1953,0,6,26
vkVSCC7xljjrAI4UGfnKEQ,0,2016-05-28,Super simple place but amazing nonetheless. It...,5,AEx2SYEUJmTxVVB18LlCwA,bv2nCi5Qv5vroFiqKGopiw,0,0,QC,4.0,84,0,6,26
n6QzIUObkYshz4dz2QRJTw,0,2016-05-28,Small unassuming place that changes their menu...,5,VR6GpWIda3SfvPC-lg9H3w,bv2nCi5Qv5vroFiqKGopiw,0,0,QC,4.5,50,0,6,26
MV3CcKScW05u5LVfF6ok0g,0,2016-05-28,Lester's is located in a beautiful neighborhoo...,5,CKC0-MOWMqoeWf6s-szl8g,bv2nCi5Qv5vroFiqKGopiw,0,0,QC,4.0,70,0,6,26
IXvOzsEMYtiJI0CARmj77Q,0,2016-05-28,Love coming here. Yes the place always needs t...,4,ACFtxLv8pGrrxMm6EgjreA,bv2nCi5Qv5vroFiqKGopiw,0,0,QC,3.5,61,0,6,26


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score